# RNNと文章のクラス分類

この節では文章のクラス分類を扱います。

例えばニュースのジャンル分類やレビュー文章のポジネガ分類などに応用可能です。

なお、時系列の分類問題は一般に系列ラベリングと呼ばれます。

## IMDbレビューデータセット

IMDbはアマゾン社で運営されているレビューサイトであり、レビューは0から10までのスコアがつけられます。

ここからスタンフォード大学の研究者らが50000件のレビューを抽出し文章のポジネガ分析のベンチマークデータセットとして公開しています。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

imdb.vocabはこのレビューに登場しているすべての単語を事前に抽出したボキャブラリーファイルです。

train/posには訓練用のポジティブなレビューのテキストファイルが大量に入っていてほかも同様です

In [22]:
import glob
import pathlib
import re
import torch
from torch import nn, optim
from torch.utils.data import (DataLoader, Dataset, TensorDataset)
import tqdm

In [5]:
remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

In [6]:
def text2ids(text, vocab_dict):
  text = remove_marks_regex.sub("", text)
  text = shift_marks_regex.sub(r" \1 ", text)
  tokens = text.split()

  return [vocab_dict.get(token, 0) for token in tokens]

In [7]:
def list2tensor(token_idxes, max_len=100, padding=True):
  if len(token_idxes) > max_len:
    token_idxes = token_idxes[:max_len]
  n_tokens = len(token_idxes)
  if padding:
    token_idxes = token_idxes + [0] * (max_len - n_tokens)
  return torch.tensor(token_idxes, dtype=torch.int64), n_tokens

text2idsは長い文字列をトークンIDのリストに変換する関数です。

list2tensorはIDのリストをint64のTensorに変換する関数です。

### Datasetクラスの作成

この2つの関数を使用して次のようにDatasetクラスを作ります。

コンストラクタ内でテキストファイルのパスとラベルをまとめたTupleのリストをつくり、__getitem__ないでそのファイルを実際に読み取ってTensorに変換しているのがポイントです。

Tensorはmax_lenで指定される長さにパディングされて統一されるので、その後の扱いが容易になります。

また、0でパディングする前のもともとの長さもn_tokensも後で必要ですのでいっしょに返します。



In [19]:
class IMDBDataset(Dataset):
  def __init__(self, dir_path, train=True,
                    max_len=100, padding=True):
    self.max_len = max_len
    self.padding = padding
    
    path = pathlib.Path(dir_path)
    vocab_path = path.joinpath("imdb.vocab")
    
    # ボキャブラリファイルを読み込み、行ごとに分割
    self.vocab_array = vocab_path.open() \
                        .read().strip().splitlines()
    # 単語をキーとし、値がIDのdictを作る
    self.vocab_dict = dict((w, i+1) \
        for (i, w) in enumerate(self.vocab_array))
    if train:
        target_path = path.joinpath("train")
    else:
        target_path = path.joinpath("test")
    pos_files = sorted(glob.glob(
        str(target_path.joinpath("pos/*.txt"))))
    neg_files = sorted(glob.glob(
        str(target_path.joinpath("neg/*.txt"))))
    # posは1, negは0のlabelを付けて
    # (file_path, label)のtupleのリストを作成
    self.labeled_files = \
        list(zip([0]*len(neg_files), neg_files )) + \
        list(zip([1]*len(pos_files), pos_files))
  
  @property
  def vocab_size(self):
    return len(self.vocab_array)  
  def __len__(self):
    return len(self.labeled_files)  
  def __getitem__(self, idx):
    label, f = self.labeled_files[idx]
    # ファイルのテキストデータを読み取って小文字に変換
    data = open(f).read().lower()
    # テキストデータをIDのリストに変換
    data = text2ids(data, self.vocab_dict)
    # IDのリストをTensorに変換
    data, n_tokens = list2tensor(data, self.max_len, self.padding)
    return data, label, n_tokens

### 訓練用とテスト用のDataLoaderの作成

あとはこれまでの章と同様にこれを利用して訓練用とテスト用のDataLoaderを作成します。

In [20]:
train_data = IMDBDataset("../data/aclImdb/")
test_data = IMDBDataset("../data/aclImdb/", train=False)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)


In [23]:
class SequenceTaggingNet(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim,
                                padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x, h0=None, l=None):
        # IDをEmbeddingで多次元のベクトルに変換する
        # xは(batch_size, step_size) 
        # -> (batch_size, step_size, embedding_dim)
        x = self.emb(x)
        # 初期状態h0と共にRNNにxを渡す
        # xは(batch_size, step_size, embedding_dim)
        # -> (batch_size, step_size, hidden_dim)
        x, h = self.lstm(x, h0)
        # 最後のステップのみ取り出す
        # xは(batch_size, step_size, hidden_dim)
        # -> (batch_size, 1)
        if l is not None:
            # 入力のもともとの長さがある場合はそれを使用する
            x = x[list(range(len(x))), l-1, :]
        else:
            # なければ単純に最後を使用する
            x = x[:, -1, :]
        # 取り出した最後のステップを線形層に入れる
        x = self.linear(x)
        # 余分な次元を削除する
        # (batch_size, 1) -> (batch_size, )
        x = x.squeeze()
        return x

In [24]:
def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y, l in data_loader:
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        with torch.no_grad():
            y_pred = net(x, l=l)
            y_pred = (y_pred > 0).long()
            ys.append(y)
            ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [25]:
from statistics import mean

# num_embeddingsには0を含めてtrain_data.vocab_size+1を入れる
net = SequenceTaggingNet(train_data.vocab_size+1, num_layers=2)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        l = l.to("cuda:0")
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, "cuda:0")
    val_acc = eval_net(net, test_loader, "cuda:0")
    print(epoch, mean(losses), train_acc, val_acc)

100%|██████████| 782/782 [00:03<00:00, 244.63it/s]


0 0.6837702416398032 0.5338799953460693 0.5325199961662292


100%|██████████| 782/782 [00:03<00:00, 259.18it/s]


1 0.6856625672344052 0.5612399578094482 0.5541599988937378


100%|██████████| 782/782 [00:03<00:00, 259.26it/s]


...

7 0.3305620495944529 0.8900399804115295 0.7666800022125244


100%|██████████| 782/782 [00:03<00:00, 246.70it/s]


8 0.28193727903582555 0.8029199838638306 0.7033999562263489


100%|██████████| 782/782 [00:03<00:00, 259.68it/s]


9 0.2369297416142338 0.9423199892044067 0.7879999876022339


In [26]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

train_X, train_y = load_svmlight_file(
    "../data/aclImdb/train/labeledBow.feat")
test_X, test_y = load_svmlight_file(
    "../data/aclImdb/test/labeledBow.feat",
    n_features=train_X.shape[1])

model = LogisticRegression(C=0.1, max_iter=1000)
model.fit(train_X, train_y)
model.score(train_X, train_y), model.score(test_X, test_y)

/home/tk/university/zemi/tokai_teio/inspections/pytorch/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.93124, 0.39392)

In [27]:
class SequenceTaggingNet2(SequenceTaggingNet):

    def forward(self, x, h0=None, l=None):
        # IDをEmbeddingで多次元のベクトルに変換
        x = self.emb(x)
        
        # 長さ情報が与えられている場合はPackedSequenceを作る
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        
        # RNNに通す
        x, h = self.lstm(x, h0)
        
        # 最後のステップを取り出して線形層に入れる
        if l is not None:
            # 長さ情報がある場合は最後の層の
            # 内部状態のベクトルを直接利用できる
            # LSTMは通常の内部状態の他にブロックセルの状態も
            # あるので内部状態のみを使用する
            hidden_state, cell_state = h
            x = hidden_state[-1]
        else:
            x = x[:, -1, :]
            
        # 線形層に入れる
        x = self.linear(x).squeeze()
        return x

In [28]:
for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        # 長さの配列を長い順にソート
        l, sort_idx = torch.sort(l, descending=True)
        # 得られたインデクスを使用してx,yも並べ替え
        x = x[sort_idx]
        y = y[sort_idx]
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, "cuda:0")
    val_acc = eval_net(net, test_loader, "cuda:0")
    print(epoch, mean(losses), train_acc, val_acc)

100%|██████████| 782/782 [00:03<00:00, 243.43it/s]


0 0.20585377895228013 0.9509599804878235 0.7823999524116516


100%|██████████| 782/782 [00:03<00:00, 244.14it/s]


1 0.162170249618628 0.9549999833106995 0.7743600010871887


100%|██████████| 782/782 [00:03<00:00, 245.02it/s]


...

7 0.05059656970467313 0.9889199733734131 0.7694399952888489


100%|██████████| 782/782 [00:03<00:00, 245.28it/s]


8 0.05079126190942 0.9908799529075623 0.770359992980957


100%|██████████| 782/782 [00:03<00:00, 246.23it/s]


9 0.05038726589490024 0.9933199882507324 0.7657999992370605
